In [1]:
!pip install flask-ngrok imbalanced-learn openpyxl

from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
import joblib

app = Flask(__name__)
run_with_ngrok(app)  # Esto expone el servidor con ngrok

@app.route("/entrenar", methods=["POST"])
def entrenar_modelo():
    data = request.get_json()
    registros = data["registros"]
    df = pd.DataFrame(registros)

    # Asegurarse de que las columnas sean las correctas y estén limpias
    if "is_assigned_automatic" not in df.columns:
        return jsonify({"error": "Falta columna objetivo"}), 400

    # Preprocesamiento (resumen simplificado)
    df = df.fillna(0)
    Y = df["is_assigned_automatic"]
    X = df.drop(columns=["is_assigned_automatic"])

    # Balanceo
    smote = SMOTE(random_state=42)
    X_resampled, Y_resampled = smote.fit_resample(X, Y)

    # Escalar
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_resampled)

    # Modelo
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation="relu", input_shape=(X_train.shape[1],)),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dense(1, activation="sigmoid")
    ])
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    model.fit(X_train, Y_resampled, epochs=5, batch_size=32, verbose=1)

    # Guardar modelo (opcional)
    model.save("modelo_mlp.h5")
    return jsonify({"mensaje": "Modelo entrenado exitosamente."})

app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-8:
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.11/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/urllib3/connectionpool.py", line 787, in urlopen
    response = self._make_request(
            